---
title: Data Wrangling with Polars
subtitle: |
  | PyData Northern Utah
  | [github.com/marcdotson/pydata-polars](https://github.com/marcdotson/pydata-polars)
author: Marc Dotson
date: 2024-11-20
title-slide-attributes:
  data-background-color: "#486790"
format: 
  revealjs:
    theme: marc.scss     # Modified simple theme.
    slide-number: c/t    # Numbered slides current/total.
    self-contained: true # Render to a single HTML file.
execute:
  eval: false
  echo: true
jupyter: python3
---


# Why Polars?

## 

:::: {.columns .v-center}

::: {.column width="40%"}
![](../figures/polars.png){fig-align="center"}
:::

::: {.column width="60%"}
::: {.incremental}
- Newer open source library for data wrangling
- Alternative to Pandas for manipulating DataFrames
- **Fast** -- written in Rust, uses Apache Arrow, built to parallelize and use GPUs, allows for lazy evaluation
- More consistent syntax than Pandas
- Anagram of its query engine (OLAP) and Rust (rs)
:::
:::

::::

# Using Polars

## Load Packages and Data

After installing with `pip install polars`, we can import polars and read in data.

::: {.panel-tabset .scrollable}
## Python


In [ ]:
#| eval: true
import polars as pl

customer_data = pl.read_csv('data/customer_data.csv')
customer_data

## Pandas

In [ ]:
import pandas as pd

customer_data = pd.read_csv('data/customer_data.csv')
customer_data

## R


```{r}
library(tidyverse)

customer_data <- read_csv("data/customer_data.csv")
customer_data
```

:::

## Filter Observations

Polars **DataFrames** have methods that are similar to {dplyr} since they're both mirroring SQL. DataFrames are composed of columns called **Series** (i.e., equivalent to vectors). Unlike pandas DataFrames, Polars DataFrames don't have a **row index**.

We would reference column names with `data['column_name']` (like R's `data$column_name` or `data["column_name"]` or just `column_name` with tidyeval), but Polars allows for `pl.col('column_name')`.

We use quotation marks for every column name.

::: {.panel-tabset .scrollable}
## Python


In [ ]:
customer_data.filter(pl.col('college_degree') == 'Yes')
customer_data.filter(pl.col('region') != 'West')
customer_data.filter(pl.col('gender') != 'Female', pl.col('income') > 70000)

## R


```{r}
filter(customer_data, college_degree == "Yes")

filter(customer_data, region != "West")

filter(customer_data, gender == "Female", income > 70000)
```

:::

## Slice Observations

Python is **zero-indexed**. This is probably the most problematic (and very computer science-based) difference and why it's nice to avoid indexing if you can!

The parameters for Polars' `.slice()` are the start index and the length.

::: {.panel-tabset .scrollable}
## Python


In [ ]:
customer_data.slice(0, 5)

## R


```{r}
slice(customer_data, 1:5)
```

:::

## Arrange Observations

It can be strange at first, but **namespacing** is critical.

- A function is preceded by the package name (e.g., `pl.col()`), unless you import the specific function (e.g., `from polars import col`).
- A method is preceded by an object name of a certain type (e.g., `customer_data.sort()`).
- Since object types are tied to packages, the chain back to the corresponding package is always present, explicitly or implicitly.

Note that its `True` and `False`, not `TRUE` and `FALSE` or `true` and `false`.

::: {.panel-tabset .scrollable}
## Python


In [ ]:
customer_data.sort(pl.col('birth_year'))
customer_data.sort(pl.col('birth_year'), descending=True)

## R


```{r}
arrange(customer_data, birth_year)

arrange(customer_data, desc(birth_year))
```

:::

## Select Variables

Using single square brackets `[ ]` creates a **list**. This is similar to creating a vector in R with `c()`. A list is a fundamental Python object type and can be turned into a Series.

Function (and method) arguments are also called **parameters**.

- Some parameters are **positional** that have to be specified in the exact position.
- Others are **keyword or named** (like in R).

::: {.panel-tabset .scrollable}
## Python


In [ ]:
customer_data.select(pl.col('region'), pl.col('review_text'))
customer_data.select(pl.col(['region', 'review_text']))

## R


```{r}
select(customer_data, region, review_text)
```

:::

## Mutate Variables

Polars is actually a query language, like SQL. So it's not surprising to see methods with names that more closely mirror queries, like the `.with_columns()` method.

::: {.panel-tabset .scrollable}
## Python


In [ ]:
customer_data.with_columns(income = pl.col('income') / 1000)

## R


```{r}
mutate(customer_data, income = income / 1000)
```

:::

## Join Data Frames

Missing values are identified as `NaN`. Series types include `str`, `bytes` (binary), `float`, `bool`, and `int`.

::: {.panel-tabset .scrollable}
## Python


In [ ]:
store_transactions = pl.read_csv('store_transactions.csv')
store_transactions.shape
store_transactions.columns

customer_data.join(store_transactions, on='customer_id', how='left')
customer_data.join(store_transactions, on='customer_id', how='inner')

## R


```{r}
store_transactions <- read_csv("store_transactions.csv")

glimpse(store_transactions)

left_join(customer_data, store_transactions, join_by(customer_id))

inner_join(customer_data, store_transactions, join_by(customer_id))
```

:::

## Consecutive Lines of Code

While possible with Python code generally, Polars embraces writing consecutive lines of code using **method chaining**. Note that:

- The entire chain needs to be surrounded with `( )`
- Each line *starts* with `.`
- You'll have to select and run the whole thing at once

::: {.panel-tabset .scrollable}
## Python


In [ ]:
(customer_data
 .join(store_transactions, on='customer_id', how='left')
 .filter(pl.col('region') == 'West', pl.col('feb_2005') == pl.col('feb_2005').max())
 .with_columns(age = 2024 - pl.col('birth_year'))
 .select(pl.col(['age', 'feb_2005']))
 .sort(pl.col('age'), descending=True)
 .slice(0, 1)
)

## R


```{r}
customer_data |> 
  left_join(store_transactions, join_by(customer_id)) |> 
  filter(region == "West", feb_2005 == max(feb_2005)) |> 
  mutate(age = 2024 - birth_year) |> 
  select(age, feb_2005) |> 
  arrange(desc(age)) |> 
  slice(1)
```

:::

## Summarize Discrete Data

The `.agg()` method stands for *aggregate*, which is exactly what `summarize()` does in R.

::: {.panel-tabset .scrollable}
## Python


In [ ]:
(customer_data
 .group_by(pl.col('region'))
 .agg(n = pl.len())
)

(customer_data
 .group_by(pl.col(['region', 'college_degree']))
 .agg(n = pl.len())
)

## R


```{r}
customer_data |> 
  count(region)

customer_data |> 
  count(region, college_degree)
```

:::

## Summarize Continuous Data

This is a good example of where object-oriented programming requires a different mindset.

- You might think that there is a general `mean()` function like in R, but there isn't and you'd have to load a specific package from the PyData stack and reference its namespace to activate such a function.
- Instead, `.mean()` is a method for Polars Series and DataFrames.

::: {.panel-tabset .scrollable}
## Python


In [ ]:
(customer_data
  .select(pl.col('income'))
  .mean()
)

(customer_data
  .select(pl.col(['income', 'credit']))
  .mean()
)

## R


```{r}
customer_data |>
  summarize(avg_income = mean(income))

customer_data |>
  summarize(
    avg_income = mean(income),
    avg_credit = mean(credit)
  )
```

:::

## Summarize Discrete and Continuous Data

We only use `.agg()` with `.group_by()`.

::: {.panel-tabset .scrollable}
## Python


In [ ]:
(customer_data
 .group_by(pl.col(['gender', 'region']))
 .agg(
   n = pl.len(), 
   avg_income = pl.col('income').mean(), 
   avg_credit = pl.col('credit').mean()
  )
 .sort(pl.col('avg_income'), descending=True)
)

## R


```{r}
customer_data |>
  group_by(gender, region) |>
  summarize(
    n = n(),
    avg_income = mean(income),
    avg_credit = mean(credit)
  ) |> 
  arrange(desc(avg_income))
```

:::

## Next Steps

- [Polars](https://docs.pola.rs) is still under development.
